Выпускная квалификационная работа по курсу «Инженер данных (Data engineer Pro)» 

Выполнила Власенкова Анна Андреевна

Задача: необходимо на основе данных с ftp.zakupki.gov.ru научиться определять группу, к которой относится контракт с кодом ОКПД-2 41, 42, 43, 71.1.

Группы могут быть следующими:

    1. Строительно-монтажные работы (СМР)
    2. Проектно-изыскательские работы (ПИР)
    3. Строительный надзор
    4. Подключение коммуникаций
    5. Прочее.

По ОКПД-2 контракты в общем случае должны разделяться так:
Строительно-монтажные работы (СМР) - 41, 42, 43(кроме нижеперечисленных)
Проектно-изыскательские работы (ПИР) - 41.1, 71.1
Подключение коммуникаций - 43.22
Строительный надзор – четкой группы нет.


Проблема: Далеко не всегда контракты указываются с нужным кодом, поэтому есть проблема как такие контракты "отловить" и определить в нужную группу.

Поэтому задача предполагает классификацию контрактов на основе объекта закупки, который сформулирован естественным языком. 
На основе этого на входе данные о контрактах. На выходе необходимо получить группу для каждого контракта.

Иногда контракт может относиться одновременно в несколько групп.

Разведочный анализ данных

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(r'fz.csv', sep=',', on_bad_lines='skip', header = None, encoding='utf-8', nrows=10000000)

C:\Users\anvla\AppData\Local\Temp\ipykernel_10736\2232484144.py:1: DtypeWarning: Columns (6,21,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'D:\ВКР\проект\fz\fz.csv', sep=',', on_bad_lines='skip', header = None, encoding='utf-8', nrows=10000000)


In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,1,1010101008321000006,211010101008301010100100100246820244,7707049388,"КРАСНОДАРСКИЙ ФИЛИАЛ ПАО \РОСТЕЛЕКОМ\""""",<НЕ ОПРЕДЕЛЕНО>,0101010083,"МО МВД РОССИИ \КОШЕХАБЛЬСКИЙ\""""",Республика Адыгея (Адыгея),18803020840390049244,...,2022-04-18,\N,<НЕ ОПРЕДЕЛЕНО>,\N,173380.50,0.00,"Изменение условий контракта, заключенного с ед...",<НЕ ОПРЕДЕЛЕНО>,Услуги по операциям с недвижимым имуществом,68.2
1,2,1010101008321000006,211010101008301010100100100246820244,7707049388,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО \РОСТЕЛЕКОМ\""""",78,0101010083,"МО МВД РОССИИ \КОШЕХАБЛЬСКИЙ\""""",Республика Адыгея (Адыгея),18803020840390049244,...,2022-04-18,\N,<НЕ ОПРЕДЕЛЕНО>,\N,173380.50,0.00,"Изменение условий контракта, заключенного с ед...",<НЕ ОПРЕДЕЛЕНО>,Услуги по операциям с недвижимым имуществом,68.2
2,3,1010101008321000007,211010101008301010100100100236820244,105076166,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ \МАН ...,01,0101010083,"МО МВД РОССИИ \КОШЕХАБЛЬСКИЙ\""""",Республика Адыгея (Адыгея),18803020840390049244,...,2022-04-19,\N,<НЕ ОПРЕДЕЛЕНО>,\N,221778.00,\N,<НЕ ОПРЕДЕЛЕНО>,<НЕ ОПРЕДЕЛЕНО>,Услуги по операциям с недвижимым имуществом,68.2
3,4,1010501691421000144,211010501691401050100101960012640323,6630008994,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ \КРУС...,66,0105016914,ГУ-РО ФОНДА СОЦИАЛЬНОГО СТРАХОВАНИЯ РОССИЙСКОЙ...,Республика Адыгея (Адыгея),39310030440139570323,...,2022-09-27,\N,<НЕ ОПРЕДЕЛЕНО>,281119.60,281119.60,0.00,<НЕ ОПРЕДЕЛЕНО>,<НЕ ОПРЕДЕЛЕНО>,"Оборудование компьютерное, электронное и оптич...",26.4
4,6,1010501691421000148,211010501691401050100101940012640323,7743763319,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ \МЕДИ...,77,0105016914,ГУ-РО ФОНДА СОЦИАЛЬНОГО СТРАХОВАНИЯ РОССИЙСКОЙ...,Республика Адыгея (Адыгея),39310030440139570323,...,\N,\N,<НЕ ОПРЕДЕЛЕНО>,120597.44,77785.15,0.00,<НЕ ОПРЕДЕЛЕНО>,<НЕ ОПРЕДЕЛЕНО>,"Оборудование компьютерное, электронное и оптич...",26.4


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 27 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   0       int64 
 1   1       int64 
 2   2       object
 3   3       int64 
 4   4       object
 5   5       object
 6   6       object
 7   7       object
 8   8       object
 9   9       object
 10  10      object
 11  11      object
 12  12      object
 13  13      object
 14  14      object
 15  15      object
 16  16      object
 17  17      object
 18  18      object
 19  19      object
 20  20      object
 21  21      object
 22  22      object
 23  23      object
 24  24      object
 25  25      object
 26  26      object
dtypes: int64(3), object(24)
memory usage: 2.0+ GB


In [6]:
df.describe()

,0,1,3
count,1.000000e+07,1.000000e+07,1.000000e+07
mean,5.917836e+06,2.608305e+18,6.875152e+10
std,3.343914e+06,7.546709e+17,3.484893e+11
min,1.000000e+00,1.193000e+16,0.000000e+00
25%,3.110704e+06,2.141700e+18,3.662998e+09
50%,5.923932e+06,2.645305e+18,6.450028e+09
75%,8.815202e+06,3.253101e+18,7.730042e+09
max,1.167939e+07,4.920456e+18,3.168901e+14


In [7]:
df.dtypes

0      int64
1      int64
2     object
3      int64
4     object
5     object
6     object
7     object
8     object
9     object
10    object
11    object
12    object
13    object
14    object
15    object
16    object
17    object
18    object
19    object
20    object
21    object
22    object
23    object
24    object
25    object
26    object
dtype: object

In [8]:
df.isna().sum()

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       1
25    1357
26    7727
dtype: int64

In [9]:
df.nunique()

0     10000000
1      7738302
2      4487568
3       513837
4       468607
5         1462
6       273719
7       147553
8         1479
9       422306
10        1794
11     3256944
12        2962
13        1472
14        2868
15        3496
16        2638
17        2461
18        3531
19        4951
20     3191689
21     4425092
22      644960
23        1060
24         353
25         304
26         625
dtype: int64

В связи с тем, что в качестве исходных данных была предложена таблица формата csv, в которой не были поименованы столбцы, выделение и именование столбцов проводилось по аналогии с шаблоном, представленным в приложении 2 Аттестационного задания.

При проведении предварительного анализа полученного датафрейма было выявлено больше двух столбцов форматы даты, которые могут иметь отношение к срокам исполнения контракта, в свзи с чем не представляется возможным точно определить продолжительность контракта.

Также было выявлено несколько столбцов, которые могут быть определены как стоимость контракта, следуя шаблону.
Так как настоящий проект носит исследовательский характер, для простоты был взят один из столбцов как подходящий для выражения значения стоимости контракта.

При реализации комерческого проекта такие детали необходимо уточнять у заказчика.

In [10]:
df_drop = df.drop([2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24], axis=1)

In [11]:
df_drop = df_drop.rename(columns={0: 'ID',
                                  1: 'CONTRACT_NUM',
                                  11: 'OBJECT_NAME',
                                  21: 'COST',
                                  25: 'DESCR',
                                  26: 'OKPD_NUM'})

In [12]:
df_drop.head()

,ID,CONTRACT_NUM,OBJECT_NAME,COST,DESCR,OKPD_NUM
0,1,1010101008321000006,<НЕ ОПРЕДЕЛЕНО>,173380.50,Услуги по операциям с недвижимым имуществом,68.2
1,2,1010101008321000006,<НЕ ОПРЕДЕЛЕНО>,173380.50,Услуги по операциям с недвижимым имуществом,68.2
2,3,1010101008321000007,<НЕ ОПРЕДЕЛЕНО>,221778.00,Услуги по операциям с недвижимым имуществом,68.2
3,4,1010501691421000144,На поставку специальных устройств для чтения «...,281119.60,"Оборудование компьютерное, электронное и оптич...",26.4
4,6,1010501691421000148,На поставку голосообразующих аппаратов для обе...,77785.15,"Оборудование компьютерное, электронное и оптич...",26.4


In [ ]:
df_drop.to_csv(r'df_drop.csv', index=False)

In [14]:
df_drop['OKPD_NUM'].value_counts().to_dict()

{21.2: 905819,
 35.1: 354452,
 35.3: 339801,
 32.5: 326758,
 10.5: 256506,
 36.0: 251014,
 61.1: 208926,
 10.1: 200553,
 10.8: 179712,
 20.5: 171742,
 19.2: 162092,
 10.3: 156261,
 1.1: 150302,
 10.7: 141070,
 22.2: 139471,
 '21.2': 134899,
 42.1: 132335,
 33.1: 121147,
 17.2: 117643,
 26.2: 116648,
 58.1: 115941,
 43.2: 112738,
 10.6: 112477,
 37.0: 103563,
 32.9: 95086,
 68.1: 93767,
 17.1: 93713,
 28.2: 91622,
 22.1: 90191,
 10.2: 84029,
 21.1: 80976,
 1.2: 80688,
 62.0: 79052,
 71.1: 76164,
 86.9: 75446,
 41.2: 74498,
 13.9: 74125,
 81.2: 73281,
 25.9: 73227,
 43.9: 72555,
 1.4: 71102,
 43.3: 67413,
 14.1: 66833,
 80.1: 65805,
 85.4: 65538,
 31.0: 62918,
 '32.5': 62724,
 10.4: 60455,
 20.4: 55400,
 56.2: 52973,
 68.2: 52953,
 35.2: 52261,
 20.1: 51729,
 26.6: 49768,
 26.5: 49547,
 71.2: 45934,
 25.7: 45715,
 38.1: 44563,
 68.3: 43960,
 53.1: 42582,
 '35.3': 42241,
 85.2: 41028,
 '35.1': 38356,
 84.2: 36717,
 20.2: 36671,
 45.2: 36394,
 42.9: 33962,
 27.4: 32967,
 49.3: 32560,
 '36.

In [21]:
df_work = df_drop.loc[(df_drop['OKPD_NUM'].isin(['41.2', '41.1', '71.1', '43.1', '43.2', '43.3', '43.9', '42.1', '42.2', '42.9', '31.0', '23.4', '13.9', '25.9'])) &(df_drop['OBJECT_NAME'] != '<НЕ ОПРЕДЕЛЕНО>')]

In [22]:
grouped_data = df_work.groupby('OKPD_NUM')['OKPD_NUM'].value_counts()
grouped_data

OKPD_NUM  OKPD_NUM
13.9      13.9         80348
23.4      23.4          9646
25.9      25.9         80766
31.0      31.0         70313
41.1      41.1          4955
41.2      41.2         81033
42.1      42.1        142023
42.2      42.2         23677
42.9      42.9         36460
43.1      43.1         13113
43.2      43.2        122717
43.3      43.3         75332
43.9      43.9         73590
71.1      71.1         82553
Name: OKPD_NUM, dtype: int64

In [23]:
df_work = df_work.drop(['CONTRACT_NUM', 'COST', 'DESCR', 'ID'], axis=1)

In [24]:
df_work['OBJECT_NAME']  = df_work['OBJECT_NAME'].astype('string')
df_work['OKPD_NUM'] = df_work['OKPD_NUM'].astype('string')
df_work.dtypes

OBJECT_NAME    string
OKPD_NUM       string
dtype: object

In [25]:
df_work.isna().sum()

OBJECT_NAME    0
OKPD_NUM       0
dtype: int64

In [26]:
df_work.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 896526 entries, 82 to 9999954
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   OBJECT_NAME  896526 non-null  string
 1   OKPD_NUM     896526 non-null  string
dtypes: string(2)
memory usage: 20.5 MB


In [28]:
df_work.to_csv(r'D:\ВКР\проект\df_work.csv', index=False)